# MCMC in Rust

In [2]:
// Get dependacies: cargo add rand indicatif plotters
:dep rand = "0.8"
:dep rand_distr = "0.4"
:dep indicatif = "0.16"
:dep plotters = "0.3"

extern crate rand;
extern crate rand_distr;
extern crate indicatif;
extern crate plotters;

use rand::Rng;
use rand_distr::{Normal, Distribution};
use indicatif::{ProgressBar, ProgressStyle};
use std::time::Instant;
use plotters::prelude::*;


fn target_distribution(x: f64) -> f64 {
    // Example target distribution: 1D standard normal distribution
    (-0.5 * x.powi(2)).exp() / (2.0 * std::f64::consts::PI).sqrt()
}

fn proposal_distribution(x: f64) -> f64 {
    // Example proposal distribution: normal distribution centered at x
    let normal = Normal::new(x, 1.0).unwrap();
    let mut rng = rand::thread_rng();
    normal.sample(&mut rng)
}

fn metropolis_hastings(target: fn(f64) -> f64, proposal: fn(f64) -> f64, initial: f64, iterations: usize) -> (Vec<f64>, std::time::Duration, f64) {
    let mut samples = Vec::new();
    samples.push(initial);
    let mut current = initial;
    let mut accepted = 0;

    let pb = ProgressBar::new(iterations as u64);
    pb.set_style(ProgressStyle::default_bar()
        .template("{spinner:.green} [{elapsed_precise}] [{bar:40.cyan/blue}] {pos}/{len} ({eta})")
        .progress_chars("#>-"));

    let start = Instant::now();

    for _ in 0..iterations {
        let proposed = proposal(current);
        let acceptance_ratio = target(proposed) / target(current);

        if rand::thread_rng().gen::<f64>() < acceptance_ratio {
            current = proposed;
            accepted += 1;
        }

        samples.push(current);
        pb.inc(1);
    }

    pb.finish_with_message("Sampling complete");

    let duration = start.elapsed();
    let acceptance_rate = accepted as f64 / iterations as f64;

    (samples, duration, acceptance_rate)
}


In [3]:
// Parameters
let initial_value = 0.0;
let num_iterations = 100000;

// Run MCMC sampler
let (samples, duration, acceptance_rate) = metropolis_hastings(target_distribution, proposal_distribution, initial_value, num_iterations);


In [4]:
// Print the time taken and acceptance rate
println!("Time taken for MCMC sampling: {:?}", duration);
println!("Acceptance rate: {:.2}%", acceptance_rate * 100.0);

Time taken for MCMC sampling: 10.865ms
Acceptance rate: 70.56%


In [5]:
// Print first 20 samples
for (i, sample) in samples.iter().take(20).enumerate() {
    println!("Sample {}: {}", i + 1, sample);
}

Sample 1: 0
Sample 2: -0.16597110362787534
Sample 3: -0.16597110362787534
Sample 4: 0.06342763525409217
Sample 5: 0.06342763525409217
Sample 6: 0.06342763525409217
Sample 7: -0.7074691252872063
Sample 8: -1.859835259617874
Sample 9: -1.859835259617874
Sample 10: -1.1883926896887416
Sample 11: -1.548456885717123
Sample 12: -1.548456885717123
Sample 13: -1.2659870090843415
Sample 14: -1.5832475085632494
Sample 15: -1.5832475085632494
Sample 16: -0.47226168363658205
Sample 17: -0.04337283321834773
Sample 18: -0.5982239722558584
Sample 19: -0.5881898757948664
Sample 20: -0.5881898757948664


()

In [ ]:
fn mean(data: &[f64]) -> f64 {
    let sum: f64 = data.iter().sum();
    let count = data.len();
    sum / count as f64
}

fn standard_deviation(data: &[f64]) -> f64 {
    let mean_value = mean(data);
    let variance: f64 = data.iter().map(|value| {
        let diff = mean_value - *value;
        diff * diff
    }).sum::<f64>() / data.len() as f64;
    variance.sqrt()
}

fn credible_interval(samples: &[f64], confidence_level: f64) -> (f64, f64) {
    let mut sorted_samples = samples.to_vec();
    sorted_samples.sort_by(|a, b| a.partial_cmp(b).unwrap());

    let lower_index = ((1.0 - confidence_level) / 2.0 * sorted_samples.len() as f64).ceil() as usize;
    let upper_index = ((1.0 + confidence_level) / 2.0 * sorted_samples.len() as f64).floor() as usize;

    (sorted_samples[lower_index], sorted_samples[upper_index])
}

Mean: -0.00791087849296739


In [8]:
// Calculate statistics
let mean_value = mean(&samples);
let sd_value = standard_deviation(&samples);
let confidence_level = 0.95;
let (lower_bound, upper_bound) = credible_interval(&samples, confidence_level);

println!("Mean: {}", mean_value);
println!("Standard Deviation: {}", sd_value);
println!("{}% credible interval: ({}, {})", confidence_level * 100.0, lower_bound, upper_bound);

Mean: -0.00791087849296739


Standard Deviation: 0.9974321878003785
95% credible interval: (-1.9460975628441455, 1.9711140382719141)


In [9]:
// Plot the trace of the samples
fn plot_trace(samples: &[f64]) -> Result<(), Box<dyn std::error::Error>> {
    let root = BitMapBackend::new("trace.png", (800, 600)).into_drawing_area();
    root.fill(&WHITE)?;

    let mut chart = ChartBuilder::on(&root)
        .caption("Trace plot of MCMC samples", ("sans-serif", 50).into_font())
        .margin(10)
        .x_label_area_size(30)
        .y_label_area_size(30)
        .build_cartesian_2d(0..samples.len(), -5.0..5.0)?;

    chart.configure_mesh().draw()?;

    chart.draw_series(LineSeries::new(
        samples.iter().enumerate().map(|(i, x)| (i, *x)),
        &BLUE,
    ))?;

    Ok(())
}

// Plot the histogram of the samples
fn plot_histogram(samples: &[f64], file_name: &str) -> Result<(), Box<dyn std::error::Error>> {
    let root = BitMapBackend::new(file_name, (640, 480)).into_drawing_area();
    root.fill(&WHITE)?;

    // Calculate histogram bins
    let n_bins = 50;
    let min_val = samples.iter().fold(f64::INFINITY, |a, &b| a.min(b));
    let max_val = samples.iter().fold(f64::NEG_INFINITY, |a, &b| a.max(b));
    let bin_width = (max_val - min_val) / n_bins as f64;
    
    let mut bins = vec![0u32; n_bins];
    for &value in samples {
        let bin = ((value - min_val) / bin_width).floor() as usize;
        if bin < n_bins {
            bins[bin] += 1;
        }
    }

    let max_count = *bins.iter().max().unwrap_or(&1) as f64;
    let normalized_bins: Vec<(f64, f64)> = bins.iter().enumerate()
        .map(|(i, &count)| {
            let x = min_val + (i as f64 + 0.5) * bin_width;
            let y = count as f64 / (samples.len() as f64 * bin_width);
            (x, y)
        })
        .collect();

    let mut chart = ChartBuilder::on(&root)
        .caption("Histogram of Samples", ("sans-serif", 30).into_font())
        .margin(5)
        .x_label_area_size(30)
        .y_label_area_size(30)
        .build_cartesian_2d(
            min_val..max_val,
            0f64..normalized_bins.iter().map(|(_,y)| *y).fold(0./0., f64::max) * 1.1
        )?;

    chart.configure_mesh().draw()?;

    chart.draw_series(
        normalized_bins.iter().map(|&(x, y)| {
            Rectangle::new(
                [(x - bin_width/2.0, 0.0), (x + bin_width/2.0, y)],
                BLUE.filled(),
            )
        }),
    )?;

    Ok(())
}

// Plot the density of the samples
fn plot_density(samples: &[f64], file_name: &str) -> Result<(), Box<dyn std::error::Error>> {
    let root = BitMapBackend::new(file_name, (800, 600)).into_drawing_area();
    root.fill(&WHITE)?;

    // Calculate kernel density estimation
    let n_points = 200;
    let bandwidth = 0.2; // Adjust this parameter for smoothing
    let min_val = samples.iter().fold(f64::INFINITY, |a, &b| a.min(b));
    let max_val = samples.iter().fold(f64::NEG_INFINITY, |a, &b| a.max(b));
    let range = max_val - min_val;
    
    let mut density_points: Vec<(f64, f64)> = Vec::with_capacity(n_points);
    for i in 0..n_points {
        let x = min_val + (i as f64 / n_points as f64) * range;
        let density = samples.iter().fold(0.0, |sum, &sample| {
            sum + (-0.5 * ((x - sample) / bandwidth).powi(2)).exp()
        }) / (samples.len() as f64 * bandwidth * (2.0 * std::f64::consts::PI).sqrt());
        density_points.push((x, density));
    }

    let max_density = density_points.iter().map(|(_, y)| *y).fold(0./0., f64::max);

    let mut chart = ChartBuilder::on(&root)
        .caption("Density Plot of Samples", ("sans-serif", 30).into_font())
        .margin(5)
        .x_label_area_size(30)
        .y_label_area_size(40)
        .build_cartesian_2d(
            min_val..max_val,
            0f64..max_density * 1.1
        )?;

    chart.configure_mesh()
        .x_desc("Value")
        .y_desc("Density")
        .draw()?;

    // Draw the density curve
    chart.draw_series(LineSeries::new(
        density_points,
        &BLUE,
    ))?;

    Ok(())
}

In [10]:
// Save samples to a file for plotting
use std::fs::File;
use std::io::Write;
let mut file = File::create("samples.txt").unwrap();
for sample in &samples {
    writeln!(file, "{}", sample).unwrap();
}

// Plot the results
plot_trace(&samples).unwrap();
plot_histogram(&samples, "histogram.png")?;
plot_density(&samples, "density_plot.png")?;

In [15]:
fn plot_time_taken(sample_sizes: &[usize], times: &[f64], file_name: &str) -> Result<(), Box<dyn std::error::Error>> {
    let root = BitMapBackend::new(file_name, (800, 600)).into_drawing_area();
    root.fill(&WHITE)?;

    let mut chart = ChartBuilder::on(&root)
        .caption("Time Taken for MCMC Sampling", ("sans-serif", 30).into_font())
        .margin(35)
        .x_label_area_size(30)
        .y_label_area_size(50)
        .build_cartesian_2d(
            0f64..*sample_sizes.last().unwrap() as f64,
            0f64..*times.iter().max_by(|a, b| a.partial_cmp(b).unwrap()).unwrap() * 1.1
        )?;

        chart.configure_mesh()
        .x_desc("Number of Samples")
        .y_desc("Time Taken (seconds)")
        .draw()?;

    chart.draw_series(LineSeries::new(
        sample_sizes.iter().zip(times.iter()).map(|(&x, &y)| (x as f64, y)),
        &BLUE,
    ))?
    .label("Time Taken")
    .legend(|(x, y)| PathElement::new(vec![(x, y), (x + 20, y)], &BLUE));

    chart.draw_series(
        sample_sizes.iter().zip(times.iter()).map(|(&x, &y)| {
            Cross::new((x as f64, y), 5, &RED)
        })
    )?;

    chart.configure_series_labels().draw()?;

    Ok(())
}


In [16]:
// Parameters
let initial_value = 0.0;
let sample_sizes = vec![10000, 20000, 50000, 100000, 200000, 250000];
let mut times = Vec::new();

for &num_iterations in &sample_sizes {
    let (_, duration, acceptance_rate) = metropolis_hastings(target_distribution, proposal_distribution, initial_value, num_iterations);;
    times.push(duration.as_secs_f64());
    println!("Samples: {}, Time taken: {:?}, Acceptance rate: {:.2}%", num_iterations, duration, acceptance_rate * 100.0);
}

// Plot the time taken for different sample sizes
plot_time_taken(&sample_sizes, &times, "time_taken.png").unwrap();

Samples: 10000, Time taken: 999µs, Acceptance rate: 70.16%
Samples: 20000, Time taken: 1.9621ms, Acceptance rate: 70.40%
Samples: 50000, Time taken: 4.7667ms, Acceptance rate: 70.51%
Samples: 100000, Time taken: 9.6997ms, Acceptance rate: 70.60%
Samples: 200000, Time taken: 19.4163ms, Acceptance rate: 70.55%
Samples: 250000, Time taken: 24.0724ms, Acceptance rate: 70.64%


In [17]:
// Parameters
let initial_value = 0.0;
let sample_sizes = vec![10000000, 50000000, 100000000, 200000000, 500000000, 1000000000];
let mut times = Vec::new();

for &num_iterations in &sample_sizes {
    let (_, duration, acceptance_rate) = metropolis_hastings(target_distribution, proposal_distribution, initial_value, num_iterations);;
    times.push(duration.as_secs_f64());
    println!("Samples: {}, Time taken: {:?}, Acceptance rate: {:.2}%", num_iterations, duration, acceptance_rate * 100.0);
}

// Plot the time taken for different sample sizes
plot_time_taken(&sample_sizes, &times, "time_taken_2.png").unwrap();

Samples: 10000000, Time taken: 973.8369ms, Acceptance rate: 70.46%
Samples: 50000000, Time taken: 4.8246133s, Acceptance rate: 70.49%
Samples: 100000000, Time taken: 9.7106757s, Acceptance rate: 70.48%
Samples: 200000000, Time taken: 19.3779369s, Acceptance rate: 70.49%
Samples: 500000000, Time taken: 48.2593459s, Acceptance rate: 70.48%
Samples: 1000000000, Time taken: 96.2832143s, Acceptance rate: 70.48%
